# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Oct 17 2022 9:00AM,248705,10,129824,"Virtue Rx, LLC",Released
1,Oct 17 2022 8:59AM,248704,10,Enova-10714,Emerginnova,Released
2,Oct 17 2022 8:58AM,248703,12,Z10142022,"Zelenko Labs, LLC",Released
3,Oct 17 2022 8:50AM,248702,10,EYE42996,Eye Pharma Inc,Released
4,Oct 17 2022 8:50AM,248702,10,EYE42997,Eye Pharma Inc,Released
5,Oct 17 2022 8:50AM,248702,10,EYE42998,Eye Pharma Inc,Released
6,Oct 17 2022 8:50AM,248702,10,EYE42999,Eye Pharma Inc,Released
7,Oct 17 2022 8:50AM,248702,10,EYE43000,Eye Pharma Inc,Released
8,Oct 17 2022 8:50AM,248702,10,EYE43001,Eye Pharma Inc,Released
9,Oct 17 2022 8:45AM,248701,19,ACG-2102491573,ACG North America LLC,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
9,248701,Released,1
10,248702,Released,6
11,248703,Released,1
12,248704,Released,1
13,248705,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
248701,NaN,1.0
248702,NaN,6.0
248703,NaN,1.0
248704,NaN,1.0
248705,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
248690,6.0,1.0
248691,0.0,23.0
248695,9.0,2.0
248697,0.0,1.0
248698,0.0,6.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
248690,6,1
248691,0,23
248695,9,2
248697,0,1
248698,0,6


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,248690,6,1
1,248691,0,23
2,248695,9,2
3,248697,0,1
4,248698,0,6


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,248690,6,1
1,248691,,23
2,248695,9,2
3,248697,,1
4,248698,,6


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released      int32
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Oct 17 2022 9:00AM,248705,10,"Virtue Rx, LLC"
1,Oct 17 2022 8:59AM,248704,10,Emerginnova
2,Oct 17 2022 8:58AM,248703,12,"Zelenko Labs, LLC"
3,Oct 17 2022 8:50AM,248702,10,Eye Pharma Inc
9,Oct 17 2022 8:45AM,248701,19,ACG North America LLC
10,Oct 17 2022 8:30AM,248690,10,ISDIN Corporation
17,Oct 17 2022 8:30AM,248691,10,ISDIN Corporation
40,Oct 17 2022 8:26AM,248700,16,Sartorius Bioprocess Solutions
43,Oct 17 2022 8:14AM,248699,10,MedStone Pharma LLC
50,Oct 17 2022 8:06AM,248698,10,Emerginnova


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,Oct 17 2022 9:00AM,248705,10,"Virtue Rx, LLC",,1
1,Oct 17 2022 8:59AM,248704,10,Emerginnova,,1
2,Oct 17 2022 8:58AM,248703,12,"Zelenko Labs, LLC",,1
3,Oct 17 2022 8:50AM,248702,10,Eye Pharma Inc,,6
4,Oct 17 2022 8:45AM,248701,19,ACG North America LLC,,1
5,Oct 17 2022 8:30AM,248690,10,ISDIN Corporation,6,1
6,Oct 17 2022 8:30AM,248691,10,ISDIN Corporation,,23
7,Oct 17 2022 8:26AM,248700,16,Sartorius Bioprocess Solutions,,3
8,Oct 17 2022 8:14AM,248699,10,MedStone Pharma LLC,,7
9,Oct 17 2022 8:06AM,248698,10,Emerginnova,,6


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Oct 17 2022 9:00AM,248705,10,"Virtue Rx, LLC",1,
1,Oct 17 2022 8:59AM,248704,10,Emerginnova,1,
2,Oct 17 2022 8:58AM,248703,12,"Zelenko Labs, LLC",1,
3,Oct 17 2022 8:50AM,248702,10,Eye Pharma Inc,6,
4,Oct 17 2022 8:45AM,248701,19,ACG North America LLC,1,
5,Oct 17 2022 8:30AM,248690,10,ISDIN Corporation,1,6
6,Oct 17 2022 8:30AM,248691,10,ISDIN Corporation,23,
7,Oct 17 2022 8:26AM,248700,16,Sartorius Bioprocess Solutions,3,
8,Oct 17 2022 8:14AM,248699,10,MedStone Pharma LLC,7,
9,Oct 17 2022 8:06AM,248698,10,Emerginnova,6,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Oct 17 2022 9:00AM,248705,10,"Virtue Rx, LLC",1,
1,Oct 17 2022 8:59AM,248704,10,Emerginnova,1,
2,Oct 17 2022 8:58AM,248703,12,"Zelenko Labs, LLC",1,
3,Oct 17 2022 8:50AM,248702,10,Eye Pharma Inc,6,
4,Oct 17 2022 8:45AM,248701,19,ACG North America LLC,1,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,Oct 17 2022 9:00AM,248705,10,"Virtue Rx, LLC",1,NaN
1,Oct 17 2022 8:59AM,248704,10,Emerginnova,1,NaN
2,Oct 17 2022 8:58AM,248703,12,"Zelenko Labs, LLC",1,NaN
3,Oct 17 2022 8:50AM,248702,10,Eye Pharma Inc,6,NaN
4,Oct 17 2022 8:45AM,248701,19,ACG North America LLC,1,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Oct 17 2022 9:00AM,248705,10,"Virtue Rx, LLC",1,0.0
1,Oct 17 2022 8:59AM,248704,10,Emerginnova,1,0.0
2,Oct 17 2022 8:58AM,248703,12,"Zelenko Labs, LLC",1,0.0
3,Oct 17 2022 8:50AM,248702,10,Eye Pharma Inc,6,0.0
4,Oct 17 2022 8:45AM,248701,19,ACG North America LLC,1,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,1740889,45,6.0
12,248703,1,0.0
16,248700,3,0.0
19,497396,3,9.0
20,248697,1,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,1740889,45,6.0
1,12,248703,1,0.0
2,16,248700,3,0.0
3,19,497396,3,9.0
4,20,248697,1,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,45,6.0
1,12,1,0.0
2,16,3,0.0
3,19,3,9.0
4,20,1,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,45.0
1,12,Released,1.0
2,16,Released,3.0
3,19,Released,3.0
4,20,Released,1.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,16,19,20
Status,,,,,
Executing,6.0,0.0,0.0,9.0,0.0
Released,45.0,1.0,3.0,3.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,16,19,20
0,Executing,6.0,0.0,0.0,9.0,0.0
1,Released,45.0,1.0,3.0,3.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,16,19,20
0,Executing,6.0,0.0,0.0,9.0,0.0
1,Released,45.0,1.0,3.0,3.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()